# **Contrucción de Scrapper para obtener listados de precios para carros en Colombia.**
## Scrapping via busqueda de modelos de carro puntuales

### Juan Vivas 2025

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd

## Pruebas:

In [ ]:
# URL de la página de listado de vehículos
url = 'https://listado.tucarro.com.co/mercedes-a45#D[A:mercedes%20a45]'

# Realizar la solicitud HTTP a la página
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    
    # Analizar el contenido HTML de la página
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar todos los elementos que contienen información de los vehículos
    # Nota: La clase 'ui-search-result__content' es común en las páginas de TuCarro para los listados
    vehicle_elements = soup.find_all('div', class_='ui-search-result__wrapper')
    
    # Iterar sobre cada elemento de vehículo y extraer la información deseada
    for vehicle in vehicle_elements:
        # Extraer el título del vehículo
        title_element = vehicle.find('h3', class_='poly-component__title-wrapper')
        title = title_element.get_text(strip=True) if title_element else 'Título no disponible'
        
        # Extraer el precio del vehículo
        price_element = vehicle.find('span', class_='andes-money-amount__fraction')
        price = price_element.get_text(strip=True) if price_element else 'Precio no disponible'
        
        # Extraer el enlace al detalle del vehículo
        link_element = vehicle.find('h3', class_='poly-component__title-wrapper')
        link = link_element.select("a")[0]["href"] if link_element else 'Enlace no disponible'

        # Extraer el año (modelo) del vehiculo
        year_element = vehicle.find_all('li', class_='poly-attributes-list__item poly-attributes-list__separator')[0]
        year = year_element.get_text(strip=True) if year_element else 'Año no disponible'

        # Extraer el kilometraje del vehiculo
        km_element = vehicle.find_all('li', class_='poly-attributes-list__item poly-attributes-list__separator')[1]
        km = km_element.get_text(strip=True) if year_element else 'Año no disponible'

        # Extraer el kilometraje del vehiculo
        location_element = vehicle.find('span', class_='poly-component__location')
        location = location_element.get_text(strip=True) if year_element else 'Año no disponible'
        
        # Mostrar la información del vehículo
        print(f'Título: {title}')
        print(f'Precio: ${price}')
        print(f'Enlace: {link}')
        print(f'Año: {year}')
        print(f'Kilometraje: {km}')
        print(f'Ubicación: {location}\n')
        
else:
    print(f'Error al acceder a la página. Código de estado: {response.status_code}')

In [ ]:
divs = soup.find_all('div')

# Extract class names from each <div> and store them in a set to avoid duplicates
div_classes = set()
for div in divs:
    if div.has_attr('class'):  # Check if the <div> has a class attribute
        div_classes.update(div['class'])  # Update set with class names (handles multiple classes per div)

# Print all unique class names used in <div> elements
print("\nList of all unique <div> class names found:")
for class_name in sorted(div_classes):
    print(class_name)

In [ ]:
for i in vehicle_elements:
    print(i)
    break

## Construcción de Proceso:

### Codificar el input:

In [2]:
def encode_string(user_input):
    part1 = user_input.replace(" ", "-")
    
    # Codificar el input en formato URL para la segunda parte
    encoded_query = urllib.parse.quote(user_input)
    result = f"{part1}#D[A:{encoded_query}]"
    
    return result

In [3]:
carro = 'ford bronco'
url = 'https://listado.tucarro.com.co/' + encode_string(carro)

### Ping a Pagina:

In [4]:
def extract_url(url):
    # Realizar la solicitud HTTP a la página
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    else:
        raise("Can't reach url")

In [5]:
def extract_content(page):
    try:
        vehicle_elements = page.find_all('div', class_='ui-search-result__wrapper')
        return vehicle_elements
    except:
        raise("Either page or class are not accessible")

In [6]:
def get_data(elem):
    # Extraer el título del vehículo
    title_element = elem.find('h3', class_='poly-component__title-wrapper')
    title = title_element.get_text(strip=True) if title_element else 'Título no disponible'
    
    # Extraer el precio del vehículo
    price_element = elem.find('span', class_='andes-money-amount__fraction')
    price = price_element.get_text(strip=True) if price_element else 'Precio no disponible'
    
    # Extraer el enlace al detalle del vehículo
    link_element = elem.find('h3', class_='poly-component__title-wrapper')
    link = link_element.select("a")[0]["href"] if link_element else 'Enlace no disponible'

    # Extraer el año (modelo) del vehiculo
    year_element = elem.find_all('li', class_='poly-attributes-list__item poly-attributes-list__separator')[0]
    year = year_element.get_text(strip=True) if year_element else 'Año no disponible'

    # Extraer el kilometraje del vehiculo
    km_element = elem.find_all('li', class_='poly-attributes-list__item poly-attributes-list__separator')[1]
    km = km_element.get_text(strip=True) if year_element else 'Año no disponible'

    # Extraer el kilometraje del vehiculo
    location_element = elem.find('span', class_='poly-component__location')
    location = location_element.get_text(strip=True) if year_element else 'Año no disponible'

    out = {'title': title,
          'price': price,
          'year': year,
          'km': km,
          'location': location,
          'link': link}

    return out

In [7]:
def get_result(elements):

    final_result = pd.DataFrame()
    
    for elem in elements:
        out_elem = get_data(elem)

        out_df = pd.DataFrame(out_elem, index = [0])
        final_result = pd.concat([final_result, out_df])

    final_result = final_result.reset_index(drop=True)

    return final_result

### Test:

In [8]:
carro = 'bmw x6'
url = 'https://listado.tucarro.com.co/' + encode_string(carro)

In [9]:
page = extract_url(url)
elements = extract_content(page)
result = get_result(elements)

In [10]:
result

,title,price,year,km,location,link
0,Bmw X6 3.0 Xdrive35i Premium,109.000.000,2014,97.000 Km,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2794431074...
1,Bmw X6 3.0 Xdrive35i At Bi-turbo,105.000.000,2011,97.571 Km,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2800270250...
2,Bmw X6 3.0 Xdrive35i,159.997.000,2017,81.000 Km,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1533124931...
3,Bmw X6 Xdrive40i,269.000.000,2021,48.000 Km,Medellín - Antioquia,https://articulo.tucarro.com.co/MCO-2747889684...
4,Bmw X6 3.0 Xdrive40i,360.000.000,2022,34.000 Km,Tunja - Boyaca,https://articulo.tucarro.com.co/MCO-1511277231...
5,Bmw X6 M60i Xdrive,486.000.000,2024,8.086 Km,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2809077202...
6,Bmw X6 3.0 Xdrive35i,176.000.000,2018,26.225 Km,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1529895393...
7,Bmw X6 X6 Xdrive40i,331.000.000,2023,20.609 Km,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2794383688...
8,Bmw X6 Xdrive30d 3.0 Premium Diesel At 2018,172.000.000,2018,69.299 Km,Medellín - Antioquia,https://articulo.tucarro.com.co/MCO-2783120564...
9,Bmw X6 Xdrive40i,291.000.000,2022,23.612 Km,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2776100558...


In [11]:
result.link

0     https://articulo.tucarro.com.co/MCO-2794431074...
1     https://articulo.tucarro.com.co/MCO-2800270250...
2     https://articulo.tucarro.com.co/MCO-1533124931...
3     https://articulo.tucarro.com.co/MCO-2747889684...
4     https://articulo.tucarro.com.co/MCO-1511277231...
5     https://articulo.tucarro.com.co/MCO-2809077202...
6     https://articulo.tucarro.com.co/MCO-1529895393...
7     https://articulo.tucarro.com.co/MCO-2794383688...
8     https://articulo.tucarro.com.co/MCO-2783120564...
9     https://articulo.tucarro.com.co/MCO-2776100558...
10    https://articulo.tucarro.com.co/MCO-1512042839...
11    https://articulo.tucarro.com.co/MCO-2794445180...
12    https://articulo.tucarro.com.co/MCO-1536051091...
13    https://articulo.tucarro.com.co/MCO-2763539108...
14    https://articulo.tucarro.com.co/MCO-1521147659...
15    https://articulo.tucarro.com.co/MCO-2760351504...
16    https://articulo.tucarro.com.co/MCO-2800777760...
17    https://articulo.tucarro.com.co/MCO-152943